### Convolutional Neural Networks

AI Bootcamp 2022 Winter

Dates: 2022-2-7

Author: Yung-Kyun Noh

Department of Computer Science, Hanyang University


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras as keras

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train, y_test)

In [ ]:
plt.imshow(x_train[82], cmap='gray')

In [ ]:
# flatten (vectorize) for ANN
x_train_flatten = x_train.reshape(60000, 784)
x_test_flatten = x_test.reshape(10000, 784)

num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_test = keras.utils.to_categorical(y_test, num_categories)

print(y_train.shape, '\n', y_train[0])    # one-hot coding

### ANN

In [ ]:
model = Sequential()

model.add(Dense(units=512, activation='relu', input_shape=(784,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = num_categories, activation='softmax'))

model.summary()

In [ ]:
# number of the parameters in each layer
print('1st layer:', (784 + 1)*512)  # 1: bias
print('2nd layer:', (512 + 1)*512)
print('3rd layer:', (512 + 1)*10)


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_flatten, y_train,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test_flatten, y_test))


### Convolutional Neural Networks

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

In [ ]:
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(units=num_categories, activation="softmax"))
model.summary()

In [ ]:
# number of the parameters in each layer
print('1st layer:', (3*3 + 1)*75)  # 1: bias
print('3rd layer:', (58800 + 1)*10)


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

x_train_conv = x_train.reshape(-1,28,28,1)
x_test_conv = x_test.reshape(-1,28,28,1)

model.fit(x_train_conv, y_train, epochs=5, verbose=1, validation_data=(x_test_conv, y_test))


In [ ]:
model.fit(x_train_conv, y_train, epochs=5, verbose=1, validation_data=(x_test_conv, y_test))


### Deep Convolutional Neural Networks

In [ ]:
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_categories, activation="softmax"))

model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

x_train_conv = x_train.reshape(-1,28,28,1)
x_test_conv = x_test.reshape(-1,28,28,1)

model.fit(x_train_conv, y_train, epochs=20, verbose=1, validation_data=(x_test_conv, y_test))

### Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.05,  # Randomly zoom image
    width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)

batch_size = 32
img_iter = datagen.flow(x_train_conv, y_train, batch_size=batch_size)
datagen.fit(x_train_conv)

In [ ]:
model.fit(img_iter,
          epochs=20,
          steps_per_epoch=len(x_train)/batch_size, # Run same number of steps we would if we were not using a generator.
          validation_data=(x_test_conv, y_test))

In [ ]:
# no augmentation
model.fit(x_train_conv, y_train, epochs=20, verbose=1, validation_data=(x_test_conv, y_test))

In [ ]:
# show the first five filters
filters, biases = model.layers[0].get_weights()
print(filters.shape)
plt.subplot(151)
plt.imshow(filters[:,:,0,0], cmap='gray')
plt.subplot(152)
plt.imshow(filters[:,:,0,1], cmap='gray')
plt.subplot(153)
plt.imshow(filters[:,:,0,2], cmap='gray')
plt.subplot(154)
plt.imshow(filters[:,:,0,3], cmap='gray')
plt.subplot(155)
plt.imshow(filters[:,:,0,4], cmap='gray')
plt.show()
